In [1]:
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install mpld3
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import json
import os


[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#first, a list with the more relevant names of the columns is createad
infocol = ["Territory", "Data type", "Gender", "Highest level of education attained", "Citizenship", "TIME", "Value"]
#then, the csv files are read and we use the list created before to only have information about those
df1 = (pd.read_csv("../_datasets/ISTAT_Unemployment_rate_region_2018_2022_.csv", sep=","))[infocol]
df2 = (pd.read_csv("../_datasets/ISTAT_Activity_Rate_Region_2018-2022.csv", sep=","))[infocol]

In [3]:
#here a function is defined in order to delete rows that might not interest us
def delete_row(dataframe, column_name, value_to_delete):
    filtered_dataframe = dataframe[dataframe[column_name] != value_to_delete]

    return filtered_dataframe

In [4]:
#sometimes the year value might include information about quarters, so this is another function to take only the values with 4 digits
def y_val(dataframe):
    
    dataframe['TIME'] = dataframe['TIME'].astype('str')
    mask = (dataframe['TIME'].str.len() == 4)
    dataframe= dataframe.loc[mask]
    
    return dataframe

In [5]:
#applying the year function for the first dataframe
df1 = y_val(df1)

In [6]:
df1

,Territory,Data type,Gender,Highest level of education attained,Citizenship,TIME,Value
0,Italy,unemployment rate,males,upper and post secondary,italian,2018,8.646761
5,Italy,unemployment rate,males,upper and post secondary,italian,2019,7.928640
10,Italy,unemployment rate,males,upper and post secondary,italian,2020,7.425268
15,Italy,unemployment rate,males,upper and post secondary,italian,2021,7.469522
16,Italy,unemployment rate,males,upper and post secondary,italian,2022,6.137924
...,...,...,...,...,...,...,...
11672,Nord-est,unemployment rate,males,total,foreign,2018,9.628649
11677,Nord-est,unemployment rate,males,total,foreign,2019,9.437797
11682,Nord-est,unemployment rate,males,total,foreign,2020,8.935428
11687,Nord-est,unemployment rate,males,total,foreign,2021,6.870910


In [7]:
#applying the year function for the second dataframe
df2 = y_val(df2)

In [8]:
df2

,Territory,Data type,Gender,Highest level of education attained,Citizenship,TIME,Value
0,Nord,activity rate,males,upper and post secondary,italian,2018,84.489730
1,Nord,activity rate,males,upper and post secondary,italian,2019,84.137185
2,Nord,activity rate,males,upper and post secondary,italian,2020,82.260744
3,Nord,activity rate,males,upper and post secondary,italian,2021,82.684206
4,Nord,activity rate,males,upper and post secondary,italian,2022,83.823293
...,...,...,...,...,...,...,...
9875,Mezzogiorno,activity rate,total,upper and post secondary,total,2018,62.224117
9880,Mezzogiorno,activity rate,total,upper and post secondary,total,2019,62.466245
9885,Mezzogiorno,activity rate,total,upper and post secondary,total,2020,59.330492
9890,Mezzogiorno,activity rate,total,upper and post secondary,total,2021,61.167564


In [9]:
#applying the deletion function to take out rows we're not interested
df2 = delete_row(df2, "Gender", "females")
df2 = delete_row(df2, "Gender", "males")
df2 = delete_row(df2, "Citizenship", "total")
df1 = delete_row(df1, "Gender", "females")
df1 = delete_row(df1, "Gender", "males")
df1 = delete_row(df1, "Citizenship", "total")

In [10]:
#Here we're excluding Italy because we want information only about the regions
df2 = delete_row(df2, "Territory", "Italy")
df1 = delete_row(df1, "Territory", "Italy")

In [11]:
df1

,Territory,Data type,Gender,Highest level of education attained,Citizenship,TIME,Value
275,Nord,unemployment rate,total,upper and post secondary,italian,2018,5.607093
280,Nord,unemployment rate,total,upper and post secondary,italian,2019,4.948992
285,Nord,unemployment rate,total,upper and post secondary,italian,2020,5.178384
290,Nord,unemployment rate,total,upper and post secondary,italian,2021,5.036280
291,Nord,unemployment rate,total,upper and post secondary,italian,2022,4.316984
...,...,...,...,...,...,...,...
11497,Centro (I),unemployment rate,total,"no educational degree, primary and lower secon...",italian,2018,11.549672
11502,Centro (I),unemployment rate,total,"no educational degree, primary and lower secon...",italian,2019,9.998387
11507,Centro (I),unemployment rate,total,"no educational degree, primary and lower secon...",italian,2020,10.639449
11512,Centro (I),unemployment rate,total,"no educational degree, primary and lower secon...",italian,2021,11.358337


In [12]:
#creating a pivot table to have the information of the educational leval in other columns and not rows
unemployment_df = df1.pivot_table(index=['Territory', 'Citizenship', 'TIME'], columns='Highest level of education attained', values='Value', aggfunc='mean').reset_index()

In [13]:
#renaming the new columns with the unemployment info for each educational level
unemployment_df.columns.name = None  # Remove the column name
unemployment_df = unemployment_df.rename(columns={'no educational degree, primary and lower secondary school certificate': 'UNEMP_ED_1', 'upper and post secondary': 'UNEMP_ED_2', 'tertiary (university, doctoral and specialization courses)':'UNEMP_ED_3', 'TIME':'Year'})

In [14]:
unemployment_df

,Territory,Citizenship,Year,UNEMP_ED_1,UNEMP_ED_3,total,UNEMP_ED_2
0,Centro (I),foreign,2018,15.155627,10.804081,14.453227,14.836710
1,Centro (I),foreign,2019,13.568437,11.434287,14.846058,17.514611
2,Centro (I),foreign,2020,13.153746,8.624535,12.778525,13.542032
3,Centro (I),foreign,2021,13.241853,18.102600,15.194500,16.586291
4,Centro (I),foreign,2022,12.723967,12.505329,12.980380,13.377200
5,Centro (I),italian,2018,11.356605,4.986898,8.676383,9.377720
6,Centro (I),italian,2019,9.827092,5.102061,7.718597,8.193194
7,Centro (I),italian,2020,10.372889,5.024784,7.629936,7.821124
8,Centro (I),italian,2021,11.100709,4.150024,7.694963,8.114135
9,Centro (I),italian,2022,8.462975,3.369947,6.059123,6.500167


In [15]:
#f_unemp = unemployment_df.loc[unemployment_df['Citizenship'] == 'foreign']
#f_unemp

In [16]:
#result = df1.loc[df1['Citizenship'] == 'foreign']
#result

In [17]:
#creating a pivot table to have the information of the educational leval in other columns and not rows
activity_df = df2.pivot_table(index=['Territory', 'Citizenship', 'TIME'], columns='Highest level of education attained', values='Value', aggfunc='mean').reset_index()

In [18]:
#renaming the new columns with the unemployment info for each educational level
activity_df.columns.name = None  # Remove the column name
activity_df = activity_df.rename(columns={'no educational degree, primary and lower secondary school certificate': 'ACT_ED_1', 'upper and post secondary': 'ACT_ED_2', 'tertiary (university, doctoral and specialization courses)':'ACT_ED_3', 'TIME':'Year'})

In [19]:
activity_df

,Territory,Citizenship,Year,ACT_ED_1,ACT_ED_3,total,ACT_ED_2
0,Centro (I),foreign,2018,70.985618,77.124190,73.572533,75.708602
1,Centro (I),foreign,2019,70.615605,75.937080,73.539525,76.776958
2,Centro (I),foreign,2020,66.399470,68.671068,68.004095,70.110227
3,Centro (I),foreign,2021,66.442788,71.376057,69.614376,73.098578
4,Centro (I),foreign,2022,68.150298,74.444015,70.789589,72.925446
5,Centro (I),italian,2018,52.889158,83.546888,67.794273,71.418533
6,Centro (I),italian,2019,52.150330,83.869037,67.642946,71.001206
7,Centro (I),italian,2020,49.991570,82.418189,66.140077,69.852723
8,Centro (I),italian,2021,50.522212,83.899395,66.848048,70.399843
9,Centro (I),italian,2022,51.609045,83.923159,67.995490,71.828835


In [20]:
#after making sure there's mostly numerical numbers in both dataframes we make the merge
DF5 = pd.merge(unemployment_df, activity_df, on=['Territory', 'Citizenship', 'Year'], how='outer')

Merge Unemployment rate x Activity rage (by educational level)

In [21]:
DF5

,Territory,Citizenship,Year,UNEMP_ED_1,UNEMP_ED_3,total_x,UNEMP_ED_2,ACT_ED_1,ACT_ED_3,total_y,ACT_ED_2
0,Centro (I),foreign,2018,15.155627,10.804081,14.453227,14.836710,70.985618,77.124190,73.572533,75.708602
1,Centro (I),foreign,2019,13.568437,11.434287,14.846058,17.514611,70.615605,75.937080,73.539525,76.776958
2,Centro (I),foreign,2020,13.153746,8.624535,12.778525,13.542032,66.399470,68.671068,68.004095,70.110227
3,Centro (I),foreign,2021,13.241853,18.102600,15.194500,16.586291,66.442788,71.376057,69.614376,73.098578
4,Centro (I),foreign,2022,12.723967,12.505329,12.980380,13.377200,68.150298,74.444015,70.789589,72.925446
5,Centro (I),italian,2018,11.356605,4.986898,8.676383,9.377720,52.889158,83.546888,67.794273,71.418533
6,Centro (I),italian,2019,9.827092,5.102061,7.718597,8.193194,52.150330,83.869037,67.642946,71.001206
7,Centro (I),italian,2020,10.372889,5.024784,7.629936,7.821124,49.991570,82.418189,66.140077,69.852723
8,Centro (I),italian,2021,11.100709,4.150024,7.694963,8.114135,50.522212,83.899395,66.848048,70.399843
9,Centro (I),italian,2022,8.462975,3.369947,6.059123,6.500167,51.609045,83.923159,67.995490,71.828835


In [22]:
#creating a pivot table to have the information of the educational leval in other columns and not rows
final_mashup = DF5.pivot_table(index=['Year', 'Territory'], columns='Citizenship').reset_index()

In [23]:
final_mashup

Year    Territory   ACT_ED_1              ACT_ED_2             \
Citizenship                       foreign    italian    foreign    italian   
0            2018   Centro (I)  70.985618  52.889158  75.708602  71.418533   
1            2018  Mezzogiorno  66.097724  41.577883  67.187872  59.857702   
2            2018         Nord  68.289759  55.320980  78.395477  75.863703   
3            2018     Nord-est  67.473834  55.556846  79.708734  76.656790   
4            2018   Nord-ovest  68.802661  55.156700  77.313317  75.260497   
5            2019   Centro (I)  70.615605  52.150330  76.776958  71.001206   
6            2019  Mezzogiorno  66.055395  40.642789  63.178155  60.145473   
7            2019         Nord  68.257518  55.708748  78.430684  76.024817   
8            2019     Nord-est  67.704574  55.295821  79.938751  77.447567   
9            2019   Nord-ovest  68.601709  55.998608  77.109207  74.955011   
10           2020   Centro (I)  66.399470  49.991570  70.110227  69.852723   
11           2020  Mezzogiorno  61.716367  38.809179  57.003637  57.092398   
12           2020         Nord  62.080906  53.970334  74.937094  73.921695   
13           2020     Nord-est  62.688161  54.445613  77.875076  74.922599   
14           2020   Nord-ovest  61.716206  53.640038  72.239657  73.171798   
15           2021   Centro (I)  66.442788  50.522212  73.098578  70.399843   
16           2021  Mezzogiorno  63.320470  39.703686  59.350414  58.786052   
17           2021         Nord  64.769660  54.163318  76.261093  74.061972   
18           2021     Nord-est  63.913195  54.174218  77.882568  74.410024   
19           2021   Nord-ovest  65.327416  54.156985  74.893586  73.800469   
20           2022   Centro (I)  68.150298  51.609045  72.925446  71.828835   
21           2022  Mezzogiorno  63.442347  40.331041  61.264202  59.487311   
22           2022         Nord  65.698128  54.849589  76.864817  75.355582   
23           2022     Nord-est  65.197743  55.003484  77.516503  76.074892   
24           2022   Nord-ovest  66.026004  54.741921  76.266436  74.817090   

              ACT_ED_3            UNEMP_ED_1            UNEMP_ED_2             \
Citizenship    foreign    italian    foreign    italian    foreign    italian   
0            77.124190  83.546888  15.155627  11.356605  14.836710   9.377720   
1            71.544665  74.639067  15.570274  24.215728  16.964263  17.254204   
2            76.686424  86.141796  14.491531   7.663050  12.781869   5.546833   
3            76.487557  85.724218  15.381984   6.678413  12.619294   4.849491   
4            76.827490  86.439320  13.942550   8.359876  12.920238   6.088359   
5            75.937080  83.869037  13.568437   9.827092  17.514611   8.193194   
6            65.841146  75.097647  16.924579  23.170330  19.247073  16.685488   
7            76.713407  85.910280  13.519575   7.548570  11.192228   4.881866   
8            76.943319  85.436213  14.406747   6.202183  11.210479   4.127024   
9            76.536952  86.256693  12.974600   8.479393  11.175751   5.468788   
10           68.671068  82.418189  13.153746  10.372889  13.542032   7.821124   
11           55.005406  74.371937  19.020709  21.780618  18.689590  14.901700   
12           71.332343  84.271635  12.179855   6.954854  12.133324   5.120106   
13           75.437066  83.740433  12.020518   6.316249  11.754628   4.927196   
14           68.043631  84.664571  12.277353   7.408154  12.508222   5.268204   
15           71.376057  83.899395  13.241853  11.100709  16.586291   8.114135   
16           58.843586  75.412764  20.928654  21.733578  20.727722  15.629899   
17           74.721646  85.553606  12.367512   7.227654  12.495783   4.995005   
18           77.036717  85.298496  11.021124   6.397210  10.315790   4.436290   
19           72.853186  85.742349  13.224713   7.805801  14.408137   5.419123   
20           74.444015  83.923159  12.723967   8.462975  13.377200   6.500167   
21           60.235821  75.942163  19.00020

In [24]:
final_mashup.describe()

ACT_ED_1              ACT_ED_2              ACT_ED_3             \
Citizenship    foreign    italian    foreign    italian    foreign    italian   
count        25.000000  25.000000  25.000000  25.000000  25.000000  25.000000   
mean         65.990942  51.216404  73.289484  71.064583  72.342502  83.088068   
std           2.621997   5.832111   6.574747   6.431047   6.319033   4.223537   
min          61.716206  38.809179  57.003637  57.092398  55.005406  74.371937   
25%          63.913195  50.522212  72.239657  70.399843  71.332343  83.546888   
50%          66.055395  54.156985  76.261093  73.921695  75.131896  84.664571   
75%          68.150298  55.003484  77.516503  75.260497  76.686424  85.910280   
max          70.985618  55.998608  79.938751  77.447567  78.276261  86.439320   

            UNEMP_ED_1            UNEMP_ED_2            UNEMP_ED_3             \
Citizenship    foreign    italian    foreign    italian    foreign    italian   
count        25.000000  25.000000  25.000000  25.000000  25.000000  25.000000   
mean         13.928813  10.694374  13.781459   7.673462  12.000308   4.534902   
std           2.731966   6.027526   3.350027   4.318262   3.682003   2.559704   
min          10.030182   5.598281   8.471354   3.730644   6.208408   2.209158   
25%          12.277353   6.945924  11.210479   4.881866   9.819867   3.097107   
50%          13.241853   7.805801  12.781869   5.468788  10.804081   3.260684   
75%          15.155627  11.100709  16.586291   8.193194  12.505329   5.024784   
max          20.928654  24.215728  20.727722  17.254204  19.874336  11.038761   

               total_x               total_y             
Citizenship    foreign    italian    foreign    italian  
count        25.000000  25.000000  25.000000  25.000000  
mean         13.606266   7.826844  69.518704  65.931716  
std           2.946764   4.634655   3.895783   6.922151  
min           8.828886   3.837501  59.997375  51.099076  
25%          12.051909   4.941271  67.639693  66.848048  
50%          12.883018   5.599697  70.651191  69.000339  
75%          14.846058   7.718597  72.617672  70.253738  
max          20.808093  18.566454  73.719380  71.252621

In [25]:
#result = DF5.loc[DF5['Citizenship'] == 'foreign']
#result.to_csv("../_datasets/F_MASHUP.csv")

In [26]:
#defining a list with the name of the most relevant columns
#infocols = ["LOCATION", "MEASURE", "TIME", "Value"]
#df3 = (pd.read_csv("../_datasets/OECD_Avarage_Wage.csv", sep=","))[infocols]
#df3

In [27]:
#selecting only information about Italy and deleting decimal numbers from value
df3 = df3[df3["LOCATION"]=="ITA"]
df3['Value'] = df3['Value'].astype(str)
df3['Value'] = df3['Value'].str.replace(r'\..*$', '', regex=True)
df3['Value'] = df3['Value'].astype(int)
df3

NameError: name 'df3' is not defined

In [30]:
df4 = (pd.read_csv("../_datasets/ISTAT_Unemployment_rate_region_2018_2022_.csv", sep=","))[infocol]
df4 = df4[df4["Citizenship"]=="total"]
df4 = df4[df4["Gender"]=="total"]
df4 = df4[df4["Highest level of education attained"]=="total"]
df4 = df4[df4["Territory"]=="Italy"]
df4 = y_val(df4)
tot_unemp = df4.head(5)

In [31]:
dfoecd = (pd.read_csv("../_datasets/OECD_Unemployment_Rate_2003_2023.csv", sep=","))
dfoecd = dfoecd[dfoecd['LOCATION']=="ITA"]
dfoecd = dfoecd[['TIME', 'Value']]
dfoecd.info()

FileNotFoundError: [Errno 2] No such file or directory: '../_datasets/OECD_Unemployment_Rate_2003_2023.csv'

In [ ]:
# Specify the path to the JSON file
file_path = "../_datasets/Clean/D4/net_income_by_regions_data.json"

# Load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

dfs = pd.DataFrame.from_dict(data['IT']) 
dfs = dfs.rename(columns={'TIME_PERIOD':'TIME'})
dfs['TIME'] = dfs['TIME'].astype(int)
dfs['ObsValue'] = dfs['ObsValue'].astype(np.float64)
dfs.info()

In [ ]:
DF6 = pd.merge(dfoecd, dfs, on=['TIME'], how='outer')
DF6

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a scatterplot using Seaborn
plt.figure(figsize=(10, 6))
plt.scatter(x=DF6['ObsValue'], y=DF6['Value'], label='Data Points', color='blue', marker='o')

# Add labels and a title
plt.xlabel('Annual Avarage Salary (€)')
plt.ylabel('Unemployment Rate (%)')
plt.title('Unemployment Rate x Annual Avarage Salary')

# Show the plot
plt.legend()
plt.show()


In [ ]:
# importing package 
#import matplotlib.pyplot as plt 
#import numpy as np 

# plot lines 
#plt.plot(DF6['ObsValue'], label = "line 1") 
#plt.plot(DF6['Value'], label = "line 2") 
#plt.plot(DF6['TIME'], np.sin(DF6['TIME']), label = "curve 1") 
#plt.legend() 
#plt.show()


In [29]:
import plotly as plt
import plotly.express as px
import json
from urllib.request import urlopen
import pandas as pd

In [32]:
with urlopen('https://raw.githubusercontent.com/openaccesstoimmigrants/openaccesstoimmigrants/main/_datasets/nuts_rg_60m_2013_lvl_1.geojson') as response:
 it_map = json.load(response)

it_map

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'id': 1,
   'properties': {'NUTS_ID': 'AT1',
    'STAT_LEVL_': 1,
    'SHAPE_AREA': 2.94056324358,
    'SHAPE_LEN': 9.53405997073},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[16.940278, 48.61724549899998],
      [16.949778, 48.535791999000025],
      [16.851106, 48.43863500100002],
      [16.976203, 48.172244498],
      [17.066741, 48.11868149899999],
      [17.1607975, 48.006656501],
      [17.093074, 47.708236],
      [16.421846, 47.66470449899998],
      [16.652076, 47.6229035],
      [16.64622, 47.446597],
      [16.4337615, 47.35291849999999],
      [16.508267499999988, 47.00125599900002],
      [16.113849, 46.86906799899998],
      [15.996236, 46.8353985],
      [16.121719499999983, 46.990666499999975],
      [16.015158499999984, 47.36712749899999],
      [16.171769499999982, 47.4224],
      [15.847011, 47.5677894990

In [33]:
def find_key_in_json(json_data, target_key):
    found_items = []

    # Recursive function to search for the key
    def find_key_recursive(obj, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    found_items.append({k: v})
                elif isinstance(v, (dict, list)):
                    find_key_recursive(v, key)
        elif isinstance(obj, list):
            for item in obj:
                find_key_recursive(item, key)

    find_key_recursive(json_data, target_key)
    return found_items

In [34]:
target_key = "NUTS_ID"
result = find_key_in_json(it_map, target_key)
print(result)

[{'NUTS_ID': 'AT1'}, {'NUTS_ID': 'AT2'}, {'NUTS_ID': 'AT3'}, {'NUTS_ID': 'BE3'}, {'NUTS_ID': 'BG3'}, {'NUTS_ID': 'BG4'}, {'NUTS_ID': 'CY0'}, {'NUTS_ID': 'CZ0'}, {'NUTS_ID': 'CH0'}, {'NUTS_ID': 'BE1'}, {'NUTS_ID': 'BE2'}, {'NUTS_ID': 'DE8'}, {'NUTS_ID': 'DE9'}, {'NUTS_ID': 'DEA'}, {'NUTS_ID': 'DE1'}, {'NUTS_ID': 'DE2'}, {'NUTS_ID': 'DE3'}, {'NUTS_ID': 'DE4'}, {'NUTS_ID': 'DE5'}, {'NUTS_ID': 'DE6'}, {'NUTS_ID': 'DE7'}, {'NUTS_ID': 'DEB'}, {'NUTS_ID': 'DEG'}, {'NUTS_ID': 'DK0'}, {'NUTS_ID': 'EE0'}, {'NUTS_ID': 'EL3'}, {'NUTS_ID': 'EL4'}, {'NUTS_ID': 'EL5'}, {'NUTS_ID': 'EL6'}, {'NUTS_ID': 'ES1'}, {'NUTS_ID': 'ES2'}, {'NUTS_ID': 'ES3'}, {'NUTS_ID': 'DEC'}, {'NUTS_ID': 'DED'}, {'NUTS_ID': 'DEE'}, {'NUTS_ID': 'DEF'}, {'NUTS_ID': 'FR6'}, {'NUTS_ID': 'FR7'}, {'NUTS_ID': 'FR8'}, {'NUTS_ID': 'FRA'}, {'NUTS_ID': 'HR0'}, {'NUTS_ID': 'ES4'}, {'NUTS_ID': 'ES5'}, {'NUTS_ID': 'ES6'}, {'NUTS_ID': 'ES7'}, {'NUTS_ID': 'FI1'}, {'NUTS_ID': 'FI2'}, {'NUTS_ID': 'FR1'}, {'NUTS_ID': 'FR2'}, {'NUTS_ID': 'FR3'},

In [39]:
import json

def filter_geojson_by_prefix(geojson_file, prefix):

    # Filter features based on the prefix
    filtered_features = [feature for feature in geojson_file['features'] if feature['properties']['NUTS_ID'].startswith(prefix)]

    # Create a new GeoJSON object with filtered features
    filtered_geojson = {
        "type": "FeatureCollection",
        "features": filtered_features
    }

    return filtered_geojson

# Example usage
filtered_geojson = filter_geojson_by_prefix(it_map, 'IT')
filtered_geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 1090,
   'properties': {'NUTS_ID': 'ITF',
    'STAT_LEVL_': 1,
    'SHAPE_AREA': 7.76554618345,
    'SHAPE_LEN': 20.516959983},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[17.388981, 40.89186349900001],
      [18.097453, 40.515377999],
      [18.517056, 40.135147999000026],
      [18.369068, 39.793742001],
      [18.04794750000002, 39.92847849899999],
      [17.76350449900002, 40.295507],
      [17.127098, 40.51762050000002],
      [16.867264, 40.39784549900003],
      [16.644317, 40.119095999000024],
      [16.52795550000002, 39.66865950099998],
      [17.025064499999985, 39.483578498999975],
      [17.188881, 39.020471998],
      [16.890732, 38.92712849899999],
      [16.566985499999987, 38.765481998999974],
      [16.5820845, 38.46979049700002],
      [16.0638095, 37.924418996999975],
      [15.678212, 37.95411949800001],
      [15.636063, 38.231473498000014],
      [15.652291742999978, 38.24726596

In [40]:
target_key = "NUTS_ID"
result = find_key_in_json(filtered_geojson, target_key)
print(result)

[{'NUTS_ID': 'ITF'}, {'NUTS_ID': 'ITG'}, {'NUTS_ID': 'ITH'}, {'NUTS_ID': 'ITI'}, {'NUTS_ID': 'ITC'}]


In [44]:
import json

def replace_value_in_geojson(geojson_file, key_to_replace, old_value, new_value):

    # Iterate through features and replace the value if found
    for feature in geojson_file['features']:
        if key_to_replace in feature['properties'] and feature['properties'][key_to_replace] == old_value:
            feature['properties'][key_to_replace] = new_value

# Applying function to replace values to be according to ISTAT division
replace_value_in_geojson(filtered_geojson, 'NUTS_ID', 'ITI', 'ITE')
replace_value_in_geojson(filtered_geojson, 'NUTS_ID', 'ITH', 'ITD')

In [45]:
target_key = "NUTS_ID"
result = find_key_in_json(filtered_geojson, target_key)
print(result)

[{'NUTS_ID': 'ITF'}, {'NUTS_ID': 'ITG'}, {'NUTS_ID': 'ITD'}, {'NUTS_ID': 'ITE'}, {'NUTS_ID': 'ITC'}]


In [51]:
state_id_map = {}
for feature in it_map ['features']:
 feature['id'] = feature['properties']['NUTS_ID']
 state_id_map[feature['properties']['NUTS_ID']] = feature['id']

In [52]:
df_dist = pd.read_json('https://raw.githubusercontent.com/openaccesstoimmigrants/openaccesstoimmigrants/main/_datasets/Clean/D1(b)/combined_immigrants_distribution.json')

df_dist = df_dist[df_dist["Territory"] !="IT"]

df_dist

,Territory,Year,Sex,Age,Quantity
1,ITC,2018,TOTAL,TOTAL,1656416
2,ITCD,2018,TOTAL,TOTAL,2835528
3,ITD,2018,TOTAL,TOTAL,1179112
4,ITE,2018,TOTAL,TOTAL,1222811
5,ITF,2018,TOTAL,TOTAL,592528
6,ITG,2018,TOTAL,TOTAL,232584
8,ITC,2019,TOTAL,TOTAL,1687560
9,ITCD,2019,TOTAL,TOTAL,2900417
10,ITD,2019,TOTAL,TOTAL,1212857
11,ITE,2019,TOTAL,TOTAL,1246779


In [53]:
fig = px.choropleth(
 df_dist,#soybean database
 geojson = filtered_geojson,
 locations="Territory", #define the limits on the map/geography
  #shape information
 color = "Quantity", #defining the color of the scale through the database
 hover_name = "Territory",
 hover_data =["Quantity"],
 title = "Immigrants by Region", #title of the map
 animation_frame = 'Year' #creating the application based on the year
)
fig.update_geos(fitbounds = "locations", visible = False)
fig.show()

In [54]:
!pip install dash


[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 10.4/10.4 MB 4.6 MB/s eta 0:00:00


In [58]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Immigrants distribution'),
    html.P("Select a year:"),
    dcc.RadioItems(
        id='Year', 
        options=["2018", "2019", "2020", "2021","2022"],
        value="Quantity",
        inline=True
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("Year", "Quantity"))
def display_choropleth(year):
    df = df_dist # replace with your own data source
    geojson = filtered_geojson
    fig = px.choropleth(
        df, geojson=geojson, color=year,
        locations="Territory", featureidkey="properties.NUTS_ID",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


app.run_server(debug=True)


## USE ISTAT DIVISION OF REGIONS FOR MAP

In [57]:
import plotly.express as px

df = df_dist
geojson = it_map

fig = px.choropleth(df, geojson=filtered_geojson, color="Quantity",
                    locations="Territory", featureidkey="properties.NUTS_ID",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()